In [ ]:
!pip install arxiv

In [ ]:
!pip install pypdf2

In [ ]:
!apt-get install build-essential libpoppler-cpp-dev pkg-config python3-dev
!pip install pdftotext

In [ ]:
!pip install nltk

In [ ]:
!pip install spacy

In [3]:
# Récupération des PDF
import arxiv

search = arxiv.Search(
    # Seulement les sujets IA
    query = "cat:cs.AI",
    max_results = 10,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print("Categories:", result.categories)
    print("PDF:", result.pdf_url)

ModuleNotFoundError: No module named 'arxiv'

In [ ]:
# URL READ (TEMP)

import urllib.request as libreq
from pprint import pprint

with libreq.urlopen('http://arxiv.org/pdf/2111.02916v1') as url:
    r = url.read()

#pprint(r)

In [17]:
# PDF Download

import urllib
urllib.request.urlretrieve('http://arxiv.org/pdf/2111.02916v1', "new_file.pdf")

('new_file.pdf', <http.client.HTTPMessage at 0x7f6f5d235ed0>)

In [ ]:
# PDF TO TEXT

from PyPDF2 import PdfFileReader
import pdftotext

# Extracting meta data
with open("new_file.pdf", "rb") as file:
    data = pdftotext.PDF(file)
    pdf = PdfFileReader(file)
    info = pdf.getDocumentInfo()
    number_of_pages = pdf.getNumPages()
    author = info.author
    creator = info.creator
    producer = info.producer
    subject = info.subject
    title = info.title

print("Info:", info)
print("Pages count:", number_of_pages)
print("Author:", author)
print("Creator:", creator)
print("Producer:", producer)
print("Subject:", subject)
print("Title:", title)
pdf_content = "\n".join(data)
print("Content:", pdf_content)

In [ ]:
# Bag of words

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#sentence_1="This is a good job.I will not miss it for anything"
#sentence_2="This is not good at all"

CountVec = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
#transform
#Count_data = CountVec.fit_transform([sentence_1,sentence_2])
Count_data = CountVec.fit_transform([pdf_content])

#create dataframe
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
print(cv_dataframe)

   000  02916v1  06  09548  10  1025  ...  zina  zinat  zinner  zitnik  zou  θu
0    2        1   1      1   1     1  ...     1      1       1      12    1   2

[1 rows x 1761 columns]


In [ ]:
import nltk
from nltk.tag.stanford import NERTagger
st = NERTagger('stanford-ner/all.3class.distsim.crf.ser.gz', 'stanford-ner/stanford-ner.jar')
text = pdf_content

for sent in nltk.sent_tokenize(text):
    tokens = nltk.tokenize.word_tokenize(sent)
    tags = st.tag(tokens)
    for tag in tags:
        if tag[1]=='PERSON': print(tag)

ImportError: ignored

In [6]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
nlp = en_core_web_sm.load()

In [7]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
pprint([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'),
 ('Google', 'ORG'),
 ('$5.1 billion', 'MONEY'),
 ('Wednesday', 'DATE')]


In [8]:
pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])

[(European, 'B', 'NORP'),
 (authorities, 'O', ''),
 (fined, 'O', ''),
 (Google, 'B', 'ORG'),
 (a, 'O', ''),
 (record, 'O', ''),
 ($, 'B', 'MONEY'),
 (5.1, 'I', 'MONEY'),
 (billion, 'I', 'MONEY'),
 (on, 'O', ''),
 (Wednesday, 'B', 'DATE'),
 (for, 'O', ''),
 (abusing, 'O', ''),
 (its, 'O', ''),
 (power, 'O', ''),
 (in, 'O', ''),
 (the, 'O', ''),
 (mobile, 'O', ''),
 (phone, 'O', ''),
 (market, 'O', ''),
 (and, 'O', ''),
 (ordered, 'O', ''),
 (the, 'O', ''),
 (company, 'O', ''),
 (to, 'O', ''),
 (alter, 'O', ''),
 (its, 'O', ''),
 (practices, 'O', '')]


In [9]:
from bs4 import BeautifulSoup
import requests
import re

In [10]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [22]:
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
#article = nlp(ny_bb)
article = nlp(pdf_content)
len(article.ents)

1126

In [23]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'CARDINAL': 136,
         'DATE': 257,
         'EVENT': 3,
         'FAC': 3,
         'GPE': 35,
         'LAW': 9,
         'LOC': 2,
         'NORP': 5,
         'ORDINAL': 8,
         'ORG': 134,
         'PERSON': 475,
         'PRODUCT': 20,
         'WORK_OF_ART': 39})

In [24]:
items = [x.text for x in article.ents]
Counter(items).most_common(3)

[('2021', 59), ('2020', 44), ('2018', 37)]

In [ ]:
sentences = [x for x in article.sents]
print(sentences[20])

In [ ]:
displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')
#displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

In [ ]:
#displacy.render(nlp(str(sentences[20])), style='dep', jupyter = True, options = {'distance': 120})
displacy.render(nlp(str(sentences)), style='dep', jupyter = True, options = {'distance': 120})

In [ ]:

[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(sentences)) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

In [ ]:
dict([(str(x), x.label_) for x in nlp(str(sentences)).ents])

In [ ]:
print([(x, x.ent_iob_, x.ent_type_) for x in sentences[20]])

In [40]:
displacy.render(article, jupyter=True, style='ent')